In [3]:
import torch
from torch import nn

# 2 primitives - datasets, dataloader

# DataLoader wraps an iterable around the Dataset
from torch.utils.data import DataLoader


from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
# Download train data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",    # Storage loc
    train=True,     # Downloading training dataset
    download=True,  
    transform=ToTensor()    # Without this line also same???
)

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [5]:
type(training_data)

torchvision.datasets.mnist.FashionMNIST

In [6]:
len(training_data)

60000

In [7]:
# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [8]:
batch_size = 64

# Create data loaders.
# Each iterable of dataloader will return a batch of 64 elements
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [9]:
type(train_dataloader)

torch.utils.data.dataloader.DataLoader

In [10]:
len(train_dataloader)

938

In [11]:
60000/64

937.5

In [12]:
# (N: number of items, C:Color, H:Height, W:Width)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    # print("Batch", batch)
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [19]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(28*28, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
        
    def forward(self, x):
        print("Forward func")
        x = self.flatten(x)
        # The above line makes (32, 1, 5, 5) to (32, 25)
        # Very important during sending the model training batch
        print(len(x))
        print(type(x))
        print(x[0].shape)
        return self.layers(x)

In [20]:
model = NeuralNet()
print(model)

NeuralNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layers): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)


In [21]:
# To train a model, we need a loss function and an optimizer.

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [22]:
def train(data: DataLoader, model:NeuralNet, loss_function, optimizer):
    print("Training")
    total = len(data)
    
    # Set model in "training mode" --------> What does this exactly do?
    model.train()
    
    for batch_num, (X, y) in enumerate(data):
        prediction = model(X)
        loss = loss_function(prediction, y)
        
        # Backprop
        loss.backward()     # WHY NOT model.backward(), SINCE WE ARE TRAINING THE MODEL!
                            # Because we need to compute "gradients of loss", which is not given by model, but by loss
        optimizer.step()    # AGAIN, WHY NOT MODEL.STEP()

        optimizer.zero_grad()   # https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch
        
        # if batch_num%100 == 0:
        #     print("Completed:", (batch_num/total) * 100)
            
    print()
        
        
        

In [23]:
def test(data: DataLoader, model:NeuralNet , loss_function):
    print("Evaluating")
    
    size = len(data.dataset)
    model.eval()
    test_loss, correct = 0, 0
    total_item = len(data)
    
    with torch.no_grad() :
        for batch_num, (X, y) in enumerate(data):
            pred = model(X)
            test_loss += loss_function(pred, y).item()
            
            # (pred.argmax(1) == y) :                                tensor([true, false, false, true.....])
            # (pred.argmax(1) == y).type(torch.float).sum()         tensor(27.0)
            # (pred.argmax(1) == y).type(torch.float).sum().item()  27
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            # if batch_num%10 == 0:
            #     print("Completed:", batch_num/total_item)
        
    print("Loss", test_loss/total_item)
    print("Test accuracy", 100*correct/size)

In [24]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
Training
Forward func
64
<class 'torch.Tensor'>
torch.Size([784])
Forward func
64
<class 'torch.Tensor'>
torch.Size([784])
Forward func
64
<class 'torch.Tensor'>
torch.Size([784])
Forward func
64
<class 'torch.Tensor'>
torch.Size([784])
Forward func
64
<class 'torch.Tensor'>
torch.Size([784])
Forward func
64
<class 'torch.Tensor'>
torch.Size([784])
Forward func
64
<class 'torch.Tensor'>
torch.Size([784])
Forward func
64
<class 'torch.Tensor'>
torch.Size([784])
Forward func
64
<class 'torch.Tensor'>
torch.Size([784])
Forward func
64
<class 'torch.Tensor'>
torch.Size([784])
Forward func
64
<class 'torch.Tensor'>
torch.Size([784])
Forward func
64
<class 'torch.Tensor'>
torch.Size([784])
Forward func
64
<class 'torch.Tensor'>
torch.Size([784])
Forward func
64
<class 'torch.Tensor'>
torch.Size([784])
Forward func
64
<class 'torch.Tensor'>
torch.Size([784])
Forward func
64
<class 'torch.Tensor'>
torch.Size([784])
Forward func
64
<class 'torch.Tensor'>


KeyboardInterrupt: 